# ✈️ FlightAI Assistant: LLM-Powered Airline Support with Tool Integration  

## 🌟 Overview  
This notebook demonstrates a **FlightAI** conversational assistant 🤖 powered by a local LLM (Llama3.2 via Ollama) with **function calling** capabilities. The system provides real-time flight price information through a user-friendly chat interface.  

---

## 🚀 Key Features  
- **💬 Interactive Chat Interface**: Built with Gradio for seamless customer interactions  
- **� Tool Integration**: Dynamically calls `get_ticket_price` when users ask about flight prices  
- **✂️ Concise Responses**: Configured for 1-sentence answers (short & accurate)  
- **🗂️ Predefined Pricing Data**: Supports major cities (London, Paris, Tokyo, Berlin)  

---

## ⚙️ How It Works  
1. **🔍 Detection**: LLM identifies price-related queries (e.g., *"How much to Paris?"*)  
2. **� Tool Call**: Triggers `get_ticket_price` with the destination city  
3. **💰 Data Fetch**: Retrieves price from a predefined dictionary  
4. **💡 Smart Response**: LLM delivers answer with the fetched price  

---

## 📋 Example Queries  
- *"How much is a ticket to Paris?"* 🇫🇷  
- *"What's the price for flights to Tokyo?"* 🇯🇵  
- *"Cost of a ticket to Berlin?"* 🇩🇪  

---

## 🛠️ Technical Stack  
- **LLM**: Llama3.2 (via Ollama) 🦙  
- **Tools**: Custom function calling with JSON schema 📜  
- **UI**: Gradio chat interface 🖥️  
- **Data**: Hardcoded pricing dictionary 💵  

---

## ℹ️ Usage Notes  
- ✅ Admits when it doesn’t know an answer ❓  
- 🔠 Case-insensitive cities (e.g., *"LONDON" = "london"*)  
- ⚠️ Designed for **single-city** inquiries (no multi-city comparisons)  

In [1]:
import os
from dotenv import load_dotenv
from anthropic import Anthropic
from openai import OpenAI
import gradio as gr
import json

In [2]:
MODEL = "llama3.2"
llama_client = OpenAI(base_url = 'http://localhost:11434/v1', api_key='ollama')

In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [4]:
def chat(message,history):
    messages = [{'role':'system','content':system_message}] + history + [{'role':'user','content':message}]
    response = llama_client.chat.completions.create(
        model = MODEL,
        messages = messages
    )
    return response.choices[0].message.content

In [5]:
gr.ChatInterface(fn = chat, type = 'messages').launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


# Using Tools to give information

In [6]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}
def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    print("City name is",city)
    return ticket_prices.get(city)

In [7]:
get_ticket_price("Berlin")

Tool get_ticket_price called for Berlin
City name is berlin


'$499'

In [8]:
price_function = {
    "name" : "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        'type' : 'object',
        'properties' : {
            'destination_city': {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        'required': ['destination_city'],
        'additional_properties': False
    }
}

In [9]:
tools = [{'type':'function','function':price_function}]

In [10]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [11]:
def chat(message,history):
    messages = [{'role':'system','content':system_message}] + history + [{'role':'user','content':message}]
    response = llama_client.chat.completions.create(
        model = MODEL,
        messages = messages,
        tools = tools
    )

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response,city = handle_tool_call(message)
        messages.append(message)
        #print("Message after appending with message:\n",messages)
        messages.append(response)
        #print("Messages after appending response:\n",response)
        response = llama_client.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = llama_client.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = llama_client.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [12]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
